# Transfering views to TrendMiner
In this use case, views have been exported from another tool, and need to be transferred to TrendMiner. The tags used in these views are already connected and available in TrendMiner.


## Export format
The export view format is in the following simple (json) format:

In [1]:
view_data = [
    {
        "name": "Reactor overview",
        "description": "Overview of reactor tags",
        "tags": [
            {
                "name": "TM4-BP2-CW.1",
                "color": "#d70c52"
            },
            {
                "name": "TM4-BP2-LEVEL.1",
                "color": "#346121"
            },
            {
                "name": "TM4-BP2-PRESSURE.1",
                "color": "#096096"
            }            
        ],
        "span": {
            "start": "2023-01-16 04:25:00",
            "end": "2023-01-16 05:50:00",
        },
    }
]

## Creating the client
We need to log in as the user for which the views need to be created. This user should have access to the datasources containing the tags in the views.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net/"
client_id = "wdanielsclient"
username = "wdaniels"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Brussels"
)

/Users/wouter.daniels/PycharmProjects/tm-python-interface/trendminer_interface/client.py:175: VersionMismatchWarning: This SDK version was tested for use with TrendMiner versions [2024.R1.0 | 2024.R1.1 | 2024.R2.0 | 2024.R2.1] while your TrendMiner version is [2024.R3.1-04]. Some functionality might not work as expected. 
  warnings.warn(


## View creation

There are many view customization options in TrendMiner. We can create a chart properties object to configure our views. In this case we will create stacked chart views.

In [3]:
chart_config = client.trend.chart.stacked(
    locked=True,  # locked timeframe
    grid=False, # No horizontal gridlines
    filling=False, # No filling under trendlines
    context=True, # Display context items when present
)

To keep our work organizer clean, we will put the imported views together in a folder. The following command retrieves the folder if it already exists, and otherwise creates it.

In [4]:
folder = client.folder.from_path("Imports", create_new=True)

We can now iterate over the view data to initialize the views.

In [5]:
import pandas as pd

views = []

for data in view_data:
    
    # Load tags by names
    tags = []
    for tagdata in data["tags"]:
        tag = client.tag.from_name(tagdata["name"])
        tag.color = tagdata["color"]
        tags.append(tag)
        
    # Setting the view interval.
    # We need to add the timezone information that is lacking in our data
    interval = pd.Interval(
        pd.Timestamp(data["span"]["start"], tz=client.tz),
        pd.Timestamp(data["span"]["end"], tz=client.tz),
    )

    # The context chart (bottom small overview graph) interval needs to be defined
    # Here, we simply set it to the same value is the focus chart interval
    context_interval = interval
    
    # Initialize the TrendHub view   
    thv = client.trend.view(
        name=data["name"],
        description=data["description"],
        entries=tags,
        layers=[interval],  # single layer view
        chart=chart_config,
        context_interval=context_interval,
        parent=folder,
    )
    
    views.append(thv)

The views can now be saved one by one on the server:

In [6]:
for view in views:
    view.save()

Note that after view creation, we can generate unique urls referring to each view:

In [8]:
for view in views:
    url = view.get_session_url()
    print(f"{view.name} - {url}")

Reactor overview - https://cs.trendminer.net/trendhub/#/share/45795597057d


In [9]:
# HIDDEN - item cleanup
folder.delete()